### Import libraries and dependencies

In [1]:
# Basic functionalities
import json
from pathlib import Path
import pandas as pd
import numpy as np

# datetime manipulation
import datetime as dt
import time
from time import sleep
from datetime import timedelta

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

# Deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Deep learning model persistence
from tensorflow.keras.models import model_from_json

# Graphing

import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline  




---

# Mcahine Learning - LSTM Neural Network

### Preprocessing for LSTM

Dataframe (stock_df) specified below needs to be close prices with all indicators (fundamental + technical) to be passed as features. 

In [ ]:
# Create signal dataframe as a copy
signal = stock_df.copy()

# Remove all NaN  resulting from unavailable indicator signals
signal = signal.dropna()

# Create blank row for current trading day and append to end of dataframe
signal = signal.append(pd.Series(name = signal.index.max() + timedelta(days = 1)))

# # Create target
signal['target'] = signal['close'] 

# Shift indicators to predict current trading day close
signal.iloc[:, :-1]  = signal.iloc[:, :-1].shift()

# Drop first row with NaNs resulting from data shift
signal = signal.iloc[1:, :]

# Ensure all data is ('float64')
signal = signal.astype('float')


In [ ]:
# Set features and target

X = signal.iloc[:, :-1]
y = signal['target']
# Write function to first convert X and y into arrays, train-test split according to train_proportion, and scale for LSTM processing


def scale_array(features, target, train_proportion:float = 0.8, scaler: bool = True):
    
    '''
    Prepares four arrays for training within an LSTM neural network. 
    Returns the following objects
    X_train: training set for features
    X_test: testing set for features
    y_train: training set for target(s)
    y_test: testing set for target(s)
    scaler: sklearn MinMaxScaler with fit memory required for inverse transformation of model predictions
    
    Parameters:    
    :features: Pandas dataframe or Series object containing model features
    :target: Pandas dataframe or Series object containing moel target(s)
    :train: Proportion of data to be assigned to train set. The rest will be assigned to test set.
    :scaler: Boolean. Default = True.If False, data will not be scaled.
    '''
    # Convert features and target to arrays
    X = np.array(features)
    y = np.array(target).reshape(-1,1)
    
    # Manually splitting the data
    split = int(0.8 * len(X))
    X_train = X[: split]
    X_test = X[split:]
    y_train = y[: split]
    y_test = y[split:]

    if scaler:
        # Create a MinMaxScaler object
        scaler = MinMaxScaler()

        # Fit the MinMaxScaler object with the features data X
        scaler.fit(X_train)

        # Scale the features training and testing sets
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Fit the MinMaxScaler object with the target data Y
        scaler.fit(y_train)

        # Scale the target training and testing sets
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)
    else:
        pass
    
    # Reshape the features data to pass into LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, X_test, y_train, y_test, scaler



In [ ]:
# Call scale_array on X and y to preprocess data for LSTM.

X_train, X_test, y_train, y_test, scaler = scale_array(X, y, train_proportion = 0.8)

### Build and Train the LSTM Neural Network

Below are a couple of helper functions to streamline the code. 


In [ ]:
def create_LSTM_model(
    train_set: np.ndarray,
    dropout: float = 0.2,
    layer_one_dropout: float = 0.6,
    number_layers: int = 4,
    optimizer: str = 'adam',
    loss: str = 'mean_squared_error'):
    
    '''
    Initialises a multilayer LSTM neural network, with number of units in the first layer being equal to the number of features. Number of layers is default 4, but can be specified by user.
    Each layer is accompanied by a Dropout with a rate of 0.6 for the first layer and a default of 0.2 for subsequent layers.
    After the first layer, number of units in each LSTM are reduced to 2/3 the initial size.
    '''

    # Define the LSTM RNN model.
    model = Sequential()

    # Initial model setup
    number_units = X_train.shape[1]
    number_hidden_nodes = number_units/3*2
    dropout_fraction = 0.4


    # Layer 1
    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X_train.shape[1], 1))
        )
    model.add(Dropout(0.6))

    number_units = int(number_units/3*2)

    # Intiialize layer counter
    layer_counter = 1
    
    # 'While' loop to keep adding layers until number of layers meet user specifications. Condition is "< - 1" because of need for penultimate layer not to have "return_sequences = True".
    while layer_counter < (number_layers - 1):
        # Layer 2 to n
        model.add(LSTM(units=number_units, return_sequences = True))
        model.add(Dropout(dropout_fraction))
        layer_counter+=1

    # Penultimate layer
    model.add(LSTM(units=number_units))
    model.add(Dropout(dropout_fraction))

    # Output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss)
    
    return model



### Set parameters

- The while loop below will repeatedly run the LSTM learning process given the parameters below. 
- Essentially, the cumulative returns produced by the strategy on the test data set has to meet the target cumulative returns (user-specified). Otherwise, it will run a new iteration.
- `max_iter` sets the maximum no. of iterations to run before the loop gives up, and that should hint to the user that a better model/feature input might be needed.
- `trading_threshold` sets the target returns within the specified timeframe that will trigger a trading signal from the strategy. E.g. if your trading threshold is 0.10, the strategy will only fire a trading signal if it predicts your returns will be more than 10%. 


#### Performance metrics:

- -**Loss on testing data set**
    - The loss metric on X_test, y_test
- **RMSE**
    - Root mean square error between predicted prices and actual prices
- **Cumulative returns**
    - Probably more important, given we want a trading algo to generate high returns. I've included the min/max so you can get an idea of how this changes across the testing period. If an algo produces high returns but performs really poorly at its `min`, that might indicate a risky trading strategy.


In [ ]:
# Set strategy cumulative return
strategy_cumulative_return = 0

# Set target cumulative returns as a threshold for model to achieve.
target_cumulative_return = 1.2

# Initialise model iteration counter
iter_counter = 0

# Set returns threshold for strategy to fire trading signal
trading_threshold = 0.15

# Set maximum numberof iterations to run
max_iter = 5


In [2]:
# Set strategy cumulative return
strategy_cumulative_return = 0

# Set target cumulative returns as a threshold for model to achieve.
target_cumulative_return = 1.4
    
# Initialise model iteration counter
iter_counter = 0

# Set threshold for strategy to initiate a trade
trading_threshold = 0.15

# Set maximum numberof iterations to run
max_iter = 10

# Record start time
start_time = time.time()

while strategy_cumulative_return < target_cumulative_return:
    
    # Start iteration counter
    iter_counter+=1
    
    # Create model if first iteration. Reset model if subsequent iterations
    model = create_LSTM_model(X_train,
                              dropout=0.4,
                              layer_one_dropout=0.6,
                              number_layers=4
                             )

    # Set early stopping such that each iteration stops running epochs if validation loss is not improving (i.e. minimising further)
    callback = EarlyStopping(
        monitor='val_loss',
        patience=40, mode='auto',
        restore_best_weights=False
    )
    
    # Print message to allow visual confirmation of iteration training is currently at.
    print("="*50)
    print(f"Training model iteration {iter_counter}...please wait.\n")

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=1000, batch_size=64,
        shuffle=False,
        validation_split = 0.1,  
        verbose = 0,
        callbacks = callback
    )
    
    # Print confirmation that current iteration has ended.
    print(f"Iteration {iter_counter} ended.")
   
    # Evaluate loss when predicting test data
    model_loss = model.evaluate(X_test[:-1], y_test[:-1], verbose=0)

    # Make predictions
    predicted = model.predict(X_test)
    
    # Recover the original prices instead of the scaled version
    predicted_prices = scaler.inverse_transform(predicted)
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Create a DataFrame of Real and Predicted values
    prices = pd.DataFrame({
        "Actual": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = signal.index[-len(real_prices): ]) 
    
    # Calculate actual daily returns
    prices['actual_returns'] = prices['Actual'].pct_change()
    # Create a 'last close' column
    prices['last_close'] = prices['Actual'].shift()
    # Calculate the predicted daily returns, by taking the predicted price as a proportion of the last close
    prices['predicted_returns'] = (prices['Predicted'] - prices['last_close'])/prices['last_close']

    # Actual signal = 1 if actual returns more than threshold,  -1 if less than threshold
    prices['actual_signal'] = 0
    prices.loc[prices['actual_returns'] > trading_threshold , 'actual_signal'] = 1
    prices.loc[prices['actual_returns'] < -trading_threshold , 'actual_signal'] = -1
    prices['actual_signal'].value_counts()
    
    # Strategy signal = 1 if predicted returns > threshold, -1 if less than threshold
    prices['strategy_signal'] = 0
    prices.loc[prices['predicted_returns'] > trading_threshold , 'strategy_signal'] = 1
    prices.loc[prices['predicted_returns'] < -trading_threshold , 'strategy_signal'] = -1
    prices['strategy_signal'].value_counts()
    
    # Compute strategy returns
    prices['strategy_returns'] = prices['actual_returns'] * prices['strategy_signal']
    
    # Compute strategy cumulative returns
    strategy_cumulative_return = (1+prices['strategy_returns']).cumprod()[-1]
    
    # Print performance metrics of the model given the feature weights produced by current iteration
    print(f"LSTM Method iteration {iter_counter} - performance")
    print("-"*50)
    print(f"Model loss on testing dataset: \n{model_loss:.4f}")
    print(f"Cumulative return on testing dataset: \n{strategy_cumulative_return}")
    
    if strategy_cumulative_return < target_cumulative_return:
        print(f"Target cumulative returns not met.\n")
        if iter_counter == max_iter:
            print(f"The LSTM model was not able to achieve the target cumulative returns on the testing dataset within {max_iter} iterations.\n")
            break
    elif strategy_cumulative_return >= target_cumulative_return:
        print(f"Target cumulative returns achieved\n")        
        

# Record time it took for the current iteration, and calculate average time per iteration
total_time_elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
average_time_per_iteration = time.strftime("%H:%M:%S", time.gmtime((time.time() - start_time)/iter_counter))

print(f"Total time elapsed: {total_time_elapsed}")
print(f"Average time taken to train {iter_counter} model(s): {average_time_per_iteration}")

# Calculate cumulative returns at their best and worst time points over time.
min_return = (1+prices['strategy_returns']).cumprod().min()
max_return = (1+prices['strategy_returns']).cumprod().max()

# Print cumulative return performance
print(f"From {prices.index.min()} to {prices.index.max()}, the cumulative return of the current model is {strategy_cumulative_return:.2f}.")
print(f"At its lowest, the model recorded a cumulative return of {min_return:.2f}.")
print(f"At its highest, the model recorded a cumulative return of {max_return:.2f}.")
print("="*50)


## Graphs

Once we have a model that generates the desired cumulative returns, we can print the graph for further visual confirmation that this is a suitable algo. 

Three graphs here 
- The loss metric from the training history of the eligible model
- Predicted prices vs Actual prices
- Strategy cumprod vs Actual cumprod.

In [ ]:
# Plot validation loss versus training loss

plt.plot(history.history['loss'], 'r', label='Training loss')
plt.plot(history.history['val_loss'], 'g', label='Validation loss')
plt.title('Training VS Validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:

# Plot the real vs predicted prices as a line chart
price_fig = px.line(prices, y = ['Actual', 'Predicted'],  title = "Actual vs Predicted", width= 1500, height = 600)
price_fig.show()

In [ ]:
# Plot strategy cumulative returns
strategy_cumulative_returns = (1+prices['strategy_returns']).cumprod()
actual_cumulative_returns = (1+prices['actual_returns']).cumprod()
cumulative_returns_df = pd.concat([strategy_cumulative_returns, actual_cumulative_returns], join = "inner", axis = "columns")

cumulative_returns_fig = px.line(
    cumulative_returns_df,
    y = ['strategy_returns', 'actual_returns'],
    x = cumulative_returns_df.index.values,
    title = f'Strategy  vs Actual Returns',
    width = 1500, height = 600
)

cumulative_returns_fig.show()


#### Note
Note that if we're using a threshold for buy/sell, the classification report is not very helpful. This is because e.g. if the model predicts returns more than the threshold for the current window, but returns turn out to be positive *but* less than the threshold, you'd still make $$$, but your strategy signal will be = 1 and the actual signal will be 0. 

In [ ]:
print(classification_report(prices['actual_signal'], prices['strategy_signal'], zero_division =1))



## Model Persistence (Save)

In [ ]:
# Convert model to json
model_json = model.to_json()

# Save model layout as json
file_path = Path(f"../LSTM_model_weights/{ticker}.json")
with open(file_path, "w") as json_file:
    json_file.write(model_json)

# Save weights
file_path = f"../LSTM_model_weights/{ticker}.h5"
model.save_weights(f"saved_models/{ticker}.h5")

---

## Forward Prediction and Trading Signals given saved model weights

### Model Persistence (Load)

Note that I've set up the file name here to save as (ticker_name).json.

In [4]:
# Load json and create mdoel
file_path = Path(f"./saved_models/{ticker}.json")
with open (file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# Load weights into new model
file_path = f"./saved_models/{ticker}.h5"
loaded_model.load_weights(file_path)

# Visual confirmation of model setup
print(loaded_model.summary())

In [ ]:
# Make predictions with model
predicted = loaded_model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted)